In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.express as px

def train_and_evaluate_prophet_model(df, date_column, target_column, input_month):
    # Preprocess the dataset
    df = df.dropna(thresh=len(df) * 0.25, axis=1)
    df[date_column] = pd.to_datetime(df[date_column])
    df = df.sort_values(by=date_column)
    
    # Filter the data for the input month
    df['month'] = df[date_column].dt.month
    df_month = df[df['month'] == input_month]

    # Determine the year to use for testing
    max_year = df_month[date_column].dt.year.max()
    if df_month[df_month[date_column].dt.year == max_year].empty:
        test_year = max_year - 1
    else:
        test_year = max_year

    # Split into training and test sets
    train = df_month[df_month[date_column].dt.year < test_year]
    test = df_month[df_month[date_column].dt.year == test_year]

    # Identify common columns (regressors) in both train and test sets
    common_columns = set(train.columns) & set(test.columns)
    regressor_columns = list(common_columns - {date_column, target_column, 'month'})

    # Prepare the data for Prophet
    train_prophet = train[[date_column, target_column] + regressor_columns].rename(columns={date_column: 'ds', target_column: 'y'})
    test_prophet = test[[date_column, target_column] + regressor_columns].rename(columns={date_column: 'ds', target_column: 'y'})

    # Initialize and fit the model
    model = Prophet()
    for column in regressor_columns:
        model.add_regressor(column)
    model.fit(train_prophet)

    # Make predictions
    forecast = model.predict(test_prophet.drop(columns='y'))
    mse = mean_squared_error(test_prophet['y'], forecast['yhat'])
    mae = mean_absolute_error(test_prophet['y'], forecast['yhat'])

    # Create a Plotly chart
    fig = px.line(forecast, x='ds', y='yhat', title='MAE over Time')
    fig.add_scatter(x=test_prophet['ds'], y=test_prophet['y'], mode='markers', name='Actual')

    # Analyze the impact of regressors
    regressor_importance = model.params['beta'][0][:len(regressor_columns)]
    importance_df = pd.DataFrame({'Regressor': regressor_columns, 'Importance': regressor_importance})
    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    return mse, mae, fig, importance_df

# Example usage:
# mse, mae, fig, importance_df = train_and_evaluate_prophet_model(your_dataframe, 'your_date_column', 'your_target_column', your_input_month)
# if fig is not None:
#     fig.show()


In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.express as px

def train_and_evaluate_prophet_model(df, date_column, target_column, input_month):
    # Preprocess the dataset
    df = df.dropna(thresh=len(df) * 0.25, axis=1)
    df[date_column] = pd.to_datetime(df[date_column])
    df = df.sort_values(by=date_column)
    
    # Filter the data for the input month
    df['month'] = df[date_column].dt.month
    df_month = df[df['month'] == input_month]

    # Determine the year to use for testing
    max_year = df_month[date_column].dt.year.max()
    if df_month[df_month[date_column].dt.year == max_year].empty:
        test_year = max_year - 1
    else:
        test_year = max_year

    # Split into training and test sets
    train = df_month[df_month[date_column].dt.year < test_year]
    test = df_month[df_month[date_column].dt.year == test_year]

    # Identify common columns (regressors) in both train and test sets
    common_columns = set(train.columns) & set(test.columns)
    regressor_columns = list(common_columns - {date_column, target_column, 'month'})

    # Prepare the data for Prophet
    train_prophet = train[[date_column, target_column] + regressor_columns].rename(columns={date_column: 'ds', target_column: 'y'})
    test_prophet = test[[date_column, target_column] + regressor_columns].rename(columns={date_column: 'ds', target_column: 'y'})

    # Initialize and fit the model
    model = Prophet()
    for column in regressor_columns:
        model.add_regressor(column)
    model.fit(train_prophet)

    # Make predictions
    forecast = model.predict(test_prophet.drop(columns='y'))
    forecast = forecast[['ds', 'yhat']]
    test_merged = test_prophet.merge(forecast, on='ds')
    test_merged['error'] = test_merged['y'] - test_merged['yhat']

    mse = mean_squared_error(test_merged['y'], test_merged['yhat'])
    mae = mean_absolute_error(test_merged['y'], test_merged['yhat'])

    # Create a Plotly chart showing daily error
    fig = px.bar(test_merged, x='ds', y='error', title='Daily Error in Predictions')

    # Analyze the impact of regressors
    regressor_importance = model.params['beta'][0][:len(regressor_columns)]
    importance_df = pd.DataFrame({'Regressor': regressor_columns, 'Importance': regressor_importance})
    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    return mse, mae, fig, importance_df

# Example usage:
# mse, mae, fig, importance_df = train_and_evaluate_prophet_model(your_dataframe, 'your_date_column', 'your_target_column', your_input_month)
# if fig is not None:
#     fig.show()
